In [1]:
import pymongo
from pymongo import MongoClient
import base64
import numpy as np
from PIL import Image
from io import BytesIO
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    base64_data = base64_string.split(",")[1]
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data)
    return np.array(image)

# Funktion zum Abrufen der Canvases
def get_canvases(data):
    return [decode_base64_image(canvas) for canvas in data["canvases"]]

# CNN-Modell erstellen
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binäre Klassifikation
    return model

# Alle Benutzer abrufen
users = list(collection.find({}))

# Modelle für jeden Benutzer erstellen
for user in users:
    username = user['username']
    model_path = f'models/{username}_fingerprint_model.h5'
    
    # Überprüfen, ob das Modell bereits existiert
    if os.path.exists(model_path):
        print(f"Model for {username} already exists at {model_path}")
        continue

    print(f"Creating model for {username}")

    # Positive Beispiele (Canvases des aktuellen Benutzers)
    user_canvases = get_canvases(user)
    X_positive = np.array(user_canvases)
    y_positive = np.ones(len(X_positive))

    # Negative Beispiele (Canvases der anderen Benutzer)
    X_negative = []
    y_negative = []

    for other_user in users:
        if other_user['username'] != username:
            other_user_canvases = get_canvases(other_user)
            X_negative.extend(other_user_canvases)
            y_negative.extend([0] * len(other_user_canvases))

    # Zufällig 2500 negative Beispiele auswählen
    if len(X_negative) > 2500:
        indices = np.random.choice(len(X_negative), 2500, replace=False)
        X_negative = np.array(X_negative)[indices]
        y_negative = np.array(y_negative)[indices]
    else:
        X_negative = np.array(X_negative)
        y_negative = np.array(y_negative)

    # Kombinieren der Daten
    X = np.concatenate((X_positive, X_negative), axis=0)
    y = np.concatenate((y_positive, y_negative), axis=0)

    # Aufteilen der Daten in Trainings- und Testsets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Modell erstellen und trainieren
    input_shape = X_train.shape[1:]
    model = create_cnn_model(input_shape)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Modell speichern
    try:
        model.save(model_path)
        print(f"Model for {username} saved successfully at {model_path}")
    except Exception as e:
        print(f"An error occurred while saving the model for {username}: {str(e)}")



Model for username_1 already exists at models/username_1_fingerprint_model.h5
Model for username_2 already exists at models/username_2_fingerprint_model.h5
Model for username_3 already exists at models/username_3_fingerprint_model.h5
Model for username_4 already exists at models/username_4_fingerprint_model.h5
Creating model for username_5



Epoch 1/10


110/110 [==============================] - 13s 100ms/step - loss: 9.0875 - accuracy: 0.5699 - val_loss: 0.6948 - val_accuracy: 0.5432
Epoch 2/10
110/110 [==============================] - 16s 150ms/step - loss: 0.6334 - accuracy: 0.6256 - val_loss: 0.7156 - val_accuracy: 0.5307
Epoch 3/10
110/110 [==============================] - 15s 133ms/step - loss: 0.5867 - accuracy: 0.6520 - val_loss: 0.6991 - val_accuracy: 0.5250
Epoch 4/10
110/110 [==============================] - 13s 122ms/step - loss: 0.5426 - accuracy: 0.6949 - val_loss: 0.7079 - val_accuracy: 0.5489
Epoch 5/10
110/110 [==============================] - 12s 113ms/step - l

C:\Users\Lion Witte\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
